In [1]:
import cv2
from ultralytics import YOLO

# --- Bước 1: Load model và frame đầu tiên ---
model = YOLO("E:/yolov8n.pt")

video_path = "sample_video.mp4"
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
if not ret:
    print("Không thể đọc video.")
    exit()

# --- Bước 2: Dùng chuột chọn vùng ROI ---
drawing = False
x1 = y1 = x2 = y2 = 0

def draw_rectangle(event, x, y, flags, param):
    global drawing, x1, y1, x2, y2
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        x1, y1 = x, y
    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        x2, y2 = x, y
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        x2, y2 = x, y

temp_frame = frame.copy()
cv2.namedWindow("Draw Scan Area")
cv2.setMouseCallback("Draw Scan Area", draw_rectangle)

while True:
    display = temp_frame.copy()
    if drawing or (x1 != x2 and y1 != y2):
        cv2.rectangle(display, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.imshow("Draw Scan Area", display)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cv2.destroyAllWindows()

# --- Bước 3: Detect trong vùng vừa chọn ---
x_min, y_min = min(x1, x2), min(y1, y2)
x_max, y_max = max(x1, x2), max(y1, y2)
roi = frame[y_min:y_max, x_min:x_max]

results = model(roi, verbose=False)

# --- Bước 4: Đếm các phương tiện ---
target_classes = ["car", "truck", "motorbike", "bus"]
count = 0

for box in results[0].boxes:
    cls = int(box.cls[0])
    name = model.names[cls]
    if name in target_classes:
        count += 1
        x0, y0, x1b, y1b = map(int, box.xyxy[0])
        cv2.rectangle(frame, (x0+x_min, y0+y_min), (x1b+x_min, y1b+y_min), (0, 0, 255), 2)
        cv2.putText(frame, name, (x0+x_min, y0+y_min-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

# --- Bước 5: Hiển thị kết quả ---
cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
cv2.putText(frame, f"Count: {count}", (x_min+10, y_min-10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

cv2.imshow("Detected Vehicles (Frame 1)", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

print(f"Số lượng phương tiện: {count}")

Số lượng phương tiện: 4
